In [ ]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob
import matplotlib.pyplot as plt
from datetime import datetime 
class TwitterClient(object):
    '''
    Generic Twitter Class for sentiment analysis.
    '''
    def __init__(self):
        '''
        Class constructor or initialization method.
        '''
        # keys and tokens from the Twitter Dev Console : to be requested from developer twitter: undisclosed as classified
        consumer_key = '##############'
        consumer_secret = '##############################'
        access_token = '############################'
        access_token_secret = '################################################'
 
        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")
 
    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
 
    def get_tweet_sentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'
 
    def get_tweets(self, query, count = 10):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []
 
        try:
            # call twitter api to fetch tweets
            #untill date
            
           # fetched_tweets = self.api.search(q = query, lang='en', result_type='popular', untill=2019-8-11, count = count)
           
           #duration within dates
            result=input("\nENTER SEARCH_TYPE:\npopular\nmixed\nrecent\n\n")
            datetime_from_str=input("\nENTER FROM DATE IN yy/m/d H:M:S\n\n")
            datetime_to_str=input("\nENTER TO DATE IN yy/m/d H:M:S\n\n")
            fetched_tweets = self.api.search(q = query, lang='en', result_type=result, fromDate=datetime_from_str, toDate=datetime_to_str, count = count)
            
            # parsing tweets one by one
            for tweet in fetched_tweets:
                # empty dictionary to store required params of a tweet
                parsed_tweet = {}
 
                # saving text of tweet
                parsed_tweet['text'] = tweet.text
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)
 
                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0:
                    # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)
 
            # return parsed tweets
            return tweets
 
        except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))
def main():
    # creating object of TwitterClient Class
    api = TwitterClient()
    # calling function to get tweets
    print("enter a topic:")
    my_query=input()
    s=input("Enter search key word:\n\n")
    tweets = api.get_tweets(query = s, count = 20)
    print(tweets)
    tweets = api.get_tweets(query = my_query, count = 30)
    # picking positive tweets from tweets
    ptweets = [t for t in tweets if t['sentiment'] == 'positive']
    # percentage of positive tweets    
    print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
    # picking negative tweets from tweets
    ntweets = [t for t in tweets if t['sentiment'] == 'negative']
    # percentage of negative tweets
    print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
    # percentage of neutral tweets
    print("Neutral tweets percentage: {} %".format(100*(len(tweets) - len(ntweets) - len(ptweets))/len(tweets)))
    #*******code for emoji analysis**********
   
    # printing first 5 positive tweets
    print("\n\nPositive tweets:")
    for tweet in ptweets[:10]:
        print(tweet['text'])
 
    # printing first 5 negative tweets
    print("\n\nNegative tweets:")
    for tweet in ntweets[:10]:
        print(tweet['text'])
 #pie chart 
    labels = 'Positive', 'Negative', 'Neutral'
    sizes = [100*len(ptweets)/len(tweets) ,100*len(ntweets)/len(tweets) , 100*(len(tweets) - len(ntweets) - len(ptweets))/len(tweets) ]
    colors = ['green', 'red', 'blue']
    explode = (0, 0, 0)
    plt.pie(sizes, explode=explode, labels=labels, colors=colors,autopct='%1.1f%%', shadow=True, startangle=90)
    plt.axis('equal')
    plt.show()
    #barplot
    # plot between 2 attributes 
    x = ['Positive', 'Negative', 'Neutral']
    popularity = [100*len(ptweets)/len(tweets) ,100*len(ntweets)/len(tweets) , 100*(len(tweets) - len(ntweets) - len(ptweets))/len(tweets) ]
    x_pos = [i for i, _ in enumerate(x)]

    plt.bar(x_pos, popularity, color=['green', 'red','blue'])

    plt.xlabel("Polarity")
    plt.ylabel("Percentage")
    plt.title("Public response")
    plt.xticks(x_pos, x)
# Turn on the grid
    plt.minorticks_on()
    plt.grid(which='major', linestyle='-', linewidth='0.5', color='red')
# Customize the minor grid
    plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
    plt.show()

plt.title('Sentiments')
plt.show()
if __name__ == "__main__":
    # calling main function
    main()
    